In [ ]:
import pandas as pd

# =========================
# 1) Read data
# =========================
weather_path = "../data/weather.csv"
user_path = "../cleaned_data/user_with_cnt_coupon_cleaned.csv"

weather = pd.read_csv(weather_path)
user = pd.read_csv(user_path)

weather["forecastDate"] = pd.to_datetime(weather["forecastDate"])
weather["publishDate"] = pd.to_datetime(weather["publishDate"])
user["experiment_date"] = pd.to_datetime(user["experiment_date"])

# =========================
# 2) SQL inner query 等價：
#    group by (cityName, publishDate, forecastDate)
#    rainy_period = COUNTIF(precipChance > 60)
#    rainfall_amount_mm = SUM(extracted mm from subNarrative)
# =========================

# REGEXP_EXTRACT(subNarrative, r'([\d.]+)公釐')
mm = (
    weather["subNarrative"]
    .astype(str)
    .str.extract(r"([\d.]+)公釐", expand=False)
)
weather["rainfall_amount_mm_part"] = pd.to_numeric(mm, errors="coerce").fillna(0.0)

weather["rainy_period_part"] = (weather["precipChance"] > 60).astype(int)

daily = (
    weather.groupby(["cityName", "publishDate", "forecastDate"], as_index=False)
    .agg(
        rainy_period=("rainy_period_part", "sum"),
        rainfall_amount_mm=("rainfall_amount_mm_part", "sum"),
    )
)

# =========================
# 3) SQL where 條件等價：
#    forecastDate BETWEEN target_date_start AND target_date_end
#    publishDate = target_publish_date
#
#    但我們要做成「對每個 publishDate，對應 experiment_date = publishDate + 3」
#    並用 forecastDate in [publishDate+3, publishDate+9] (共 7 天)
# =========================

daily["experiment_date"] = daily["publishDate"] + pd.Timedelta(days=3)

start = daily["experiment_date"]
end = daily["experiment_date"] + pd.Timedelta(days=6)
in_window = (daily["forecastDate"] >= start) & (daily["forecastDate"] <= end)
daily_win = daily.loc[in_window].copy()

# BigQuery DAYOFWEEK: Sunday=1 ... Saturday=7
# pandas: dayofweek Monday=0 ... Sunday=6
# weekday = Mon~Fri => 0~4 ; weekend => 5,6
dow = daily_win["forecastDate"].dt.dayofweek
daily_win["is_weekday"] = dow.between(0, 4)
daily_win["is_weekend"] = dow.isin([5, 6])

# =========================
# 4) SQL outer query 等價：
#    group by (cityName, experiment_date)
# =========================
feat = (
    daily_win.groupby(["cityName", "experiment_date"], as_index=False)
    .apply(lambda g: pd.Series({
        "rainy_day": (g["rainy_period"] > 0).sum(),
        "is_rainy_weekday": ((g["rainfall_amount_mm"] > 10) & (g["is_weekday"])).sum(),
        "is_rainy_weekend": ((g["rainfall_amount_mm"] > 10) & (g["is_weekend"])).sum(),
    }))
    .reset_index(drop=True)
)

# 你要的「一週平均」版本（比例）
feat["avg_rainy_day"] = feat["rainy_day"] / 7
feat["avg_rainy_weekday"] = feat["is_rainy_weekday"] / 5
feat["avg_rainy_weekend"] = feat["is_rainy_weekend"] / 2

# 欄位名對齊 user（cityName -> city）
feat = feat.rename(columns={"cityName": "city"})

# =========================
# 5) Left join 回 user_with_cnt_coupon_cleaned
# =========================
user_with_weather = user.merge(
    feat[["city", "experiment_date", "avg_rainy_day", "avg_rainy_weekday", "avg_rainy_weekend"]],
    on=["city", "experiment_date"],
    how="left",
)

# 先不要存檔：你可以直接看結果
print(user_with_weather.shape)
display(user_with_weather.head())
